# Fire Image Classification

## Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Keras Imports
import tensorflow as tf
import keras
from keras import backend as K
# CNN and MLP architecture
from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    UpSampling2D,
    Dropout,
    Flatten,
    BatchNormalization
)
from keras.models import Model
from keras.optimizers import SGD
from keras.initializers import RandomNormal
# Keras Callbacks
from keras.callbacks import EarlyStopping, TensorBoard
# Time Training
import time
# Data Splitting
from sklearn.model_selection import train_test_split
# Image Preprocessing
from PIL import Image
from keras.preprocessing.image import img_to_array
# Optimizing Hyperparameters
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
# Data Generator
import util
# Loading Model from JSON file
from keras.models import model_from_json

## Get the Data

In [2]:
df = pd.read_csv('Fire-Detection-Image-Dataset/fires.csv')

In [3]:
df.head()

,Unnamed: 0,Folder,filename,label
0,0,Normal Images 3,Hotel_Monterey_La_Soeur_Osaka_standard_twin_be...,0
1,1,Normal Images 3,house5.jpg,0
2,2,Normal Images 3,mi-plage-hawai.jpg,0
3,3,Normal Images 3,JB224_03_Colourful_christmas_table_setting_in_...,0
4,4,Normal Images 3,Interior Design Ideas (3).jpg,0


In [4]:
# define predictor and target sets of data
features = [df.columns[1], df.columns[2]]
target = [df.columns[3]]
X, y = df[features], df[target]
# Store number of class
num_classes = 2

## Split DataFrame Based on Label

In [5]:
def data():
    """Data providing function (will later use for Hyperas)."""
    df = pd.read_csv('Fire-Detection-Image-Dataset/fires.csv')
    # Split df into train and test, based on label
    # Credit goes to this Stack Overflow answer for the following:
    # https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
    selector = np.random.rand(len(df)) < 0.75
    df_train, df_test = df[selector], df[~selector]
    print(f"Number of Training Samples: {len(df_train)}")
    print(f"Number of Testing Samples: {len(df_test)}")
    return df_train, df_test

df_train, df_test = data()

Number of Training Samples: 466
Number of Testing Samples: 185


## Defining the Optimal Model 

### Build a Generator

For this dataset, we'll increase the efficiency of training by loading in subsections of the dataset to train on at a time, using Python-esque generator in Keras!

In [9]:
# data_gen = util.data_gen
def data_gen(df_gen, batch_size):
    """Generate batches of the dataset to train the model on, one subsection at a time.
       Credit goes to Milad Toutounchian for this implementation, originally found at:
       https://github.com/Make-School-Courses/DS-2.2-Deep-Learning/blob/master/Final_Project/image_data_prep.ipynb
    
       Parameters:
       df(DataFrame): larger portion of the datset used for training
       batch_size(int): the number of samples to include in each batch
       
       Returns:
       tuple: input features of the batch, along with corresponding labels
    
    """
    while True:
        # list of images
        x_batch = np.zeros((batch_size, 1024, 1024, 3))
        # list of labels
        y_batch = np.zeros((batch_size, 1))
        # add samples until we reach batch size
        for j in range(len(df_gen) // batch_size):
            batch_index = 0
            for index in df_gen['Unnamed: 0']:
                if batch_index < batch_size:
                    # add image to the input
                    filepath = f"Fire-Detection-Image-Dataset/{df_gen['Folder'][index]}/{df_gen['filename'][index]}"
                    img = Image.open(filepath)
                    image_red = img.resize((1024, 1024))
                    x_batch[batch_index] = img_to_array(image_red)
                    # set label
                    y_batch[batch_index] = df_gen['label'][index]
                    # increment index in the batch
                    batch_index += 1
            yield (x_batch, y_batch)

### Finding the Best Architecture

Now we'll find the optimal number of layers and neurons per layer using TensorBoard.

In [7]:
# Helper functions to Reduce Repetition of Adding Layers
def add_conv_layer(model, layer_size, needs_input):
    """Add a Keras convolutional layer to the model, along with MaxPooling.
       Will specify input shape as well if needed.
       
       Parameters:
       model(Model): Neural network in Keras
       layer_size(int): number of neurons to go in layer
       need_input(bool): signals if the convolutional layer needs to specify
                         the dimensions of the input
       
       Returns: None
       
    """
    if needs_input is True:
        # specify input dimension for 1st conv layer
        conv_layer = Conv2D(layer_size,
                            kernel_size=(3, 3),
                            activation='relu',
                            input_shape=(1024, 1024, 3))

    else:
        # otherwise all other convolutional layers don't need it
        conv_layer = Conv2D(layer_size,
                            kernel_size=(3, 3),
                            activation='relu')
    # add Convolutional layer
    model.add(conv_layer)  
    # add MaxPooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))  # no learning params
    return None
                  
    
def add_dense_layer(model, layer_size, is_output, drop_rate):
    """Add a multi-layer perceptron to the model
       Will specify 'sigmoid' for the final layer.
       
       Parameters:
       model(Model): Neural network in Keras
       layer_size(int): number of neurons to go in layer
       is_output(bdool): signals if the MLP is the last layer
       drop_rate(float): percentage of connections in Dense layer
                       to cut off
       
       Returns: None
       
    """
    # specify activation function
    activation = 'relu' if is_output is False else 'sigmoid'
    # add MLP
    model.add(Dense(layer_size, activation=activation)) 
    # Add Dropout layer and Batch Normalization
    if is_output is False:
        model.add(BatchNormalization())
        model.add(Dropout(drop_rate))
    return None


def define_model(units, conv_layers, dense_layers, dropout):
    """Define a CNN + MLP model in Keras.
    
       Parameters:
       units(int): number of neurons to go in a layer
       conv_layers(int): number of convolutional layers
       dense_layers(int): number of MLP
       dropout(float): percentage of connections in Dense layer
                       to cut off
                       
       Returns: tf.keras.Sequential: the neural network to train
    
    """
    # Instaniate model
    model = Sequential()
    # Add CNN layers
    add_conv_layer(model, units, True)
    for l in range(conv_layers - 1):
        # add convolutional layers that come after the 1st
        add_conv_layer(model, units, False)
    # Flatten the data
    model.add(Flatten())
    # Add MLP Layers
    for l in range(dense_layers - 1):
        add_dense_layer(model, units, False, dropout)
    # add final MLP, for output
    add_dense_layer(model, 1, True, dropout)
    # Compile Model
    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall()])
    # Return model
    return model

In [10]:
# Choices for the Model Architecture - values arbitrary
dense_layers = [1, 2, 3]
layer_sizes = [4, 8, 16]
conv_layers = [1, 2, 3]

# try different combinations!
# These for loops come from https://youtu.be/lV09_8432VA
for dense_layer in dense_layers:
    for size in layer_sizes:
        for conv in conv_layers:
            # name the combo
            NAME = (
                f'{conv}-conv-{size}-nodes' +
                f'-{dense_layer}-dense_layers' + 
                f'-{int(time.time())}'
            )
            # Instantiate TensorBoard to visualize model performance
            tensorboard = TensorBoard(log_dir=f'./Graph/{NAME}')
            # Define Model
            model = define_model(size, conv, dense_layer, 0.2)
            # Train the Model (using a generator!)
            epochs, batch_size = 5, 20
            history = model.fit_generator(generator=data_gen(df_train, batch_size=batch_size),
                                steps_per_epoch=len(df_train['label']) // batch_size,
                                epochs=epochs,
                                validation_data=data_gen(df_test, batch_size=batch_size),
                                validation_steps=len(df_test['label']) // batch_size, 
                                callbacks=[tensorboard])

Epoch 1/5
 7/23 [========>.....................] - ETA: 38s - loss: 0.3943 - accuracy: 0.9786 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00

KeyboardInterrupt: 

**Final Conclusion: What's the Best Architecture for the Model?**

After training the different combinations above and looking through the [logs](Graph/) returned by Tensorboard, the best model archtecture has been determined to be:

- 3 Conv2D Layers
- 1 Dense Layer
- 4 Neurons in each Layer
- Dropout Rate of 20%
- 5 epochs
- 20 samples per Batch

This model had the following metrics, which was the best of any of the other models:

- Validation Accuracy: 100%
- Validation Loss: 4.51e31%
- Validation Precision: 96.25%
- Validation Recall: 95.09%
- F1-Score: 0.9567

Although the accuracy of 100% is a little concerning, it may be possible to prevent overfitting by increase the drop out rate to 30%.

###  Finding Optimal Hyperparameters

Now, we'll find the optimal values for the hyperparameters using hyperas...


In [11]:
# Note: This cell DOES NOT work currently, I am currently debugging it. 
# I got the results below this cell from running the 'optimize.py' script in this directory.

def optimize_model():
    """Returns a dictionary with the results of the best model."""
    # define known parameters
    layer_size = 4
    # Instaniate model
    model = Sequential()
    # Instantiate TensorBoard to visualize model performance
    tensorboard = TensorBoard(log_dir='./Graph')
    # Add 3 CNN layers
    model.add(Conv2D(layer_size, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(1024, 1024, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(layer_size,
                     kernel_size=(3, 3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(layer_size,
                     kernel_size=(3, 3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Flatten the data
    model.add(Flatten())
    # Add 1 MLP Layer
    model.add(Dense(1, activation={{choice(['sigmoid', 'relu'])}}))
    # Compile Model
    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=
                      {{choice(
                          [keras.optimizers.Adadelta(),
                          'sgd', 'adam', 'rmsprop']   
                           )}},
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall()])
    # Train Model
    batch_size = {{choice([10, 20, 40])}}
    result = model.fit_generator(generator=util.data_gen(df_train,
                        batch_size=batch_size),
                        steps_per_epoch=len(df_train['label']) // batch_size,
                        epochs={{choice([3, 5, 7])}},
                        validation_data=util.data_gen(df_test, batch_size=batch_size),
                        validation_steps=len(df_test['label']) // batch_size, 
                        callbacks=[tensorboard])
    # Get Optimized results
    # get the highest validation metrics of the training epochs
    val_loss = np.amax(result.history['val_loss'])
    print(f'Best validation acc of epoch: {(1-val_loss)}')
    return {
        'loss': val_loss,
        'accuracy': 1-val_loss,
        'status': STATUS_OK,
        'model': model
    }

# find the best model!
best_run, best_model = optim.minimize(model=optimize_model,
                                    data=data,
                                    algo=tpe.suggest,
                                    max_evals=5,
                                    trials=Trials(),
                                    notebook_name='classifications')
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import seaborn as sns
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten, BatchNormalization
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import SGD
except:
    pass

try:
    from keras.initializers import RandomNormal
except:
    pass

try:
    from keras.callbacks import EarlyStopping, TensorBoard
except:
    pass

try:
    import time
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from PIL import Image
except

KeyboardInterrupt: 

**Final Conclusion: What are the Optimal Hyperparameters?**

After using Hyperas, the best hyperparameters appear to be the following (in order to balance model performance with overfitting and efficiency):

- Activation Function (for the output layer): Sigmoid
- Batch Size: 40
- Epochs: 3
- Optimizer: Adam

### Train, Summarize, and Save the Model

This will help us for future reference, so when we want to make more improvements to the model we can simply load it from a file.

In [17]:
# define the model with 4 neurons in each layer, 3 Conv layers, 1 dense, and with dropout of 20%
optimal_model = define_model(4, 3, 1, 0.2)
# Training the model
epochs, batch_size = 3, 40
history = optimal_model.fit_generator(generator=data_gen(df_train, batch_size=batch_size),
                                steps_per_epoch=len(df_train['label']) // batch_size,
                                epochs=epochs,
                                validation_data=data_gen(df_test, batch_size=batch_size),
                                validation_steps=len(df_test['label']) // batch_size, 
                                callbacks=[tensorboard])
print(optimal_model.summary())

Epoch 1/3
11/11 [==============================] - 76s 7s/step - loss: 0.3393 - accuracy: 0.9614 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 2/3
11/11 [==============================] - 90s 8s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 3/3
11/11 [==============================] - 79s 7s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 1022, 1022, 4)     112       
_________________________

### Save Model

We will define functions for both saving and loading the model (weights as well as architecture) from and Hadoop and JSON-formatted data.

The name of the files the model will be saved to shall be: 

- 'model_weights.h5' (Hadoop format)
- 'model_architecture.json' (Javascript Object Notation)

In [16]:
def save_model(model, weights_file, architecture_file):
    """Save the model weights and architecture.
    
       Parameters: 
       model(Model): keras Model object being saved
       weights_file(str): name of the Hadoop file where
                          weights will be saved
       architecture_file(str): name of the JSON file where 
                               model architecture is to be
                               saved
                               
       Returns: None
       
    """
    # Save the weights
    model.save_weights(f'{weights_file}.h5')
    # Save the architecture
    with open(f'{architecture_file}.json', 'w') as f:
        f.write(model.to_json())
    return None


def load_model(weights_file, architecture_file):
    """Read in the model weights and architecture.
    
       Parameters:
       weights_file(str): name of the Hadoop file where
                          weights loaded from
       architecture_file(str): name of the JSON file where 
                               model architecture is read from
                               
       Returns: keras.Model: new model instantiated using the 
                             information from the files
       
    """  
    # Load Architecture
    with open(f'{architecture_file}.json', 'r') as f:
        new_model = model_from_json(f.read())
    # Load Weights
    new_model.load_weights(f'{weights_file}.h5')
    return new_model

In [ ]:
# Save the model
save_model(optimized_model, 'model_weights', 'model_architecture')

## Upsample the Minority Class

We need more fire!

Look here
Upsample minority class (fire images) -> https://elitedatascience.com/imbalanced-classes?_ga=2.44533796.1624997989.1593199508-1623274989.1547664151

## Data Augumentation

Let's add more variation to the dataset now, in order to make the model more robust!

Look here: https://github.com/Make-School-Courses/DS-2.2-Deep-Learning/blob/master/Lessons/KerasforLargeDatasets.md

## Model Evaluation

Review this lesson: https://github.com/Make-School-Courses/DS-2.2-Deep-Learning/blob/master/Lessons/DeepLearningModelEvaluation.md

## Final Conclusions